<h1>Import Packages</h1>

In [101]:
# Import Packages
from __future__ import division, print_function, unicode_literals
import argparse
import h5py
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
import os

<h1>Import Loss Functions and Models</h1>
<br><br>
Here we import loss functions and model functions from 'loss.py' and 'network.py' implemented by Yang et. al., <i>Investigating Capsule Networks with Dynamic Routing for Text Classification </i> (2018). Additionally, we implemented two more model functions, namely <i>short_text_capsule_model()</i> and <i>long_text_capsule_model()</i>, based on capsule_model_B, as per the models discussed in Goldani et. al., <i>Detecting Fake News with Capsule Neural Networks</i> (2020)

In [3]:
# Import loss functions and models
from loss import spread_loss, cross_entropy, margin_loss
from network import baseline_model_kimcnn, baseline_model_cnn, capsule_model_A, capsule_model_B, short_text_capsule_model, long_text_capsule_model

Using TensorFlow backend.


<h1>Load and Preprocess LIAR Dataset</h1>
<br><br>
We now load and preprocess the LIAR dataset to prepare it for GLoVe. Reference for code: <a href = "https://github.com/KunojiLym/metis_project_4">link</a> 

In [64]:
# read the LIAR dataset

liar_credit_hist_headers = ['speaker_bt', 'speaker_f', 'speaker_ht', 'speaker_mt', 'speaker_pof']
liar_speaker_headers = ['speaker', 'speaker_job', 'speaker_us_state', 'speaker_affiliation'] + liar_credit_hist_headers
liar_column_headers = ['id', 'label', 'statement', 'subjects']  + liar_speaker_headers + ['context']

# read the pre-prepared training, validation and test sets

liar_train = pd.read_csv("./data/liar_dataset/train.tsv", sep='\t', names=liar_column_headers, index_col='id')
liar_valid = pd.read_csv("./data/liar_dataset/valid.tsv", sep='\t', names=liar_column_headers, index_col='id')
liar_test = pd.read_csv("./data/liar_dataset/test.tsv", sep='\t', names=liar_column_headers, index_col='id')

# we will use the validation set for model selection; the test set is to be left for judging the final model

liar_train

label                                          statement  \
id                                                                           
2635.json         false  Says the Annies List political group supports ...   
10540.json    half-true  When did the decline of coal start? It started...   
324.json    mostly-true  Hillary Clinton agrees with John McCain "by vo...   
1123.json         false  Health care reform legislation is likely to ma...   
9028.json     half-true  The economic turnaround started at the end of ...   
12465.json         true  The Chicago Bears have had more starting quart...   
2342.json   barely-true  Jim Dunnam has not lived in the district he re...   
153.json      half-true  I'm the only person on this stage who has work...   
5602.json     half-true  However, it took $19.5 million in Oregon Lotte...   
9741.json   mostly-true  Says GOP primary opponents Glenn Grothman and ...   
7115.json   mostly-true  For the first time in history, the share of th...   
4148.json     half-true  Since 2000, nearly 12 million Americans have s...   
5947.json         false  When Mitt Romney was governor of Massachusetts...   
8616.json   mostly-true  The economy bled $24 billion due to the govern...   
8705.json   barely-true  Most of the (Affordable Care Act) has already ...   
10683.json    half-true  In this last election in November, ... 63 perc...   
620.json           true  McCain opposed a requirement that the governme...   
3863.json   barely-true  U.S. Rep. Ron Kind, D-Wis., and his fellow Dem...   
12372.json    half-true  Water rates in Manila, Philippines, were raise...   
12385.json  mostly-true  Almost 100,000 people left Puerto Rico last year.   
10173.json        false  Women and men both are making less when you ad...   
9867.json   mostly-true  The United States has the highest corporate ta...   
12408.json  mostly-true  We just had the best year for the auto industr...   
2673.json     half-true  Says Scott Walker favors cutting up to 350,000...   
7057.json   barely-true  Says Mitt Romney wants to get rid of Planned P...   
10215.json        false              I dont know who (Jonathan Gruber) is.   
12517.json  mostly-true  Hate crimes against American Muslims and mosqu...   
3910.json     half-true  Rick Perry has never lost an election and rema...   
11092.json        false  ISIS supporter tweeted at 10:34 a.m. Shooting ...   
12163.json  mostly-true  Youth unemployment in minority communities is ...   
...                 ...                                                ...   
8119.json          true  Since the Affordable Care Act passed, 90 perce...   
1550.json          true  Debt has almost doubled in Austin under Gov. P...   
4077.json     half-true  Lets say (Republicans) take away half of our d...   
11207.json  barely-true  Theres a tremendous other number of public off...   
4216.json         false  Under last years health care reform, a bunch o...   
9365.json   barely-true  The Obama administration spent $205,075 in sti...   
981.json          false  There has been no net global warming for over ...   
11074.json    half-true  Thanks to the Obama administrations negotiatio...   
1674.json         false  This race will be the Democrats top target thi...   
6032.json     half-true  Georgia Public Service Commission member Stan ...   
7388.json   mostly-true  Says he and Mitt Romney agreed on tying minimu...   
10166.json        false  As a result of Obamacare, California seniors f...   
8371.json          true  For the first time since the Korean War, total...   
3744.json   mostly-true  Says Rick Perry turned down our invitation to ...   
8262.json   barely-true  In 2012, the state put together a list of over...   
1918.json     half-true  The Republican Party lost 1.1 million register...   
4614.json   mostly-true  The proudest accomplishment (of my tenure) was...   
13344.json   pants-fire  Recently though, the media has reported on tho...   
13239.json  barely-true  Stopped by Smil

In [6]:
liar_label_order = ['true', 'mostly-true', 'half-true', 'barely-true', 'false', 'pants-fire']

In [7]:
# Replace na with empty strings
liar_train['speaker_affiliation'].fillna('', inplace=True)
liar_valid['speaker_affiliation'].fillna('', inplace=True)
liar_test['speaker_affiliation'].fillna('', inplace=True)

liar_train['speaker_job'].fillna('', inplace=True)
liar_valid['speaker_job'].fillna('', inplace=True)
liar_test['speaker_job'].fillna('', inplace=True)

liar_train['context'].fillna('', inplace=True)
liar_valid['context'].fillna('', inplace=True)
liar_test['context'].fillna('', inplace=True)

In [8]:
# Explore data to show where inconsistent entries are present
liar_speakers_full = liar_train[liar_speaker_headers]

liar_speakers = liar_speakers_full.drop_duplicates().sort_values(by='speaker')
#liar_speakers['count_in_set'] = liar_speakers_full.groupby(liar_speaker_headers, as_index=False).size().values

liar_speakers['speaker'].value_counts()

kasim-reed                                  2
danny-tarkanian                             2
robert-puente                               2
mary-olson                                  1
mary-jordan                                 1
todd-tiahrt                                 1
pink-pony                                   1
georgia-state-road-and-tollway-authority    1
donzella-james                              1
james-florio                                1
cory-booker                                 1
christine-gilbert                           1
myra-crownover                              1
jon-kyl                                     1
bill-mccollum                               1
town-hall-audience-member                   1
candy-crowley                               1
ameripac                                    1
j-james-rohack                              1
gayle-smith                                 1
john-thune                                  1
kirk-cox                          

In [9]:
# Kasim Reed
liar_train[liar_train['speaker']=='kasim-reed'][liar_speaker_headers].drop_duplicates()

speaker     speaker_job speaker_us_state speaker_affiliation  \
id                                                                            
4659.json   kasim-reed             NaN              NaN            democrat   
13163.json  kasim-reed  Atlanta Mayor           Georgia            democrat   

            speaker_bt  speaker_f  speaker_ht  speaker_mt  speaker_pof  
id                                                                      
4659.json          1.0        0.0         5.0         7.0          1.0  
13163.json         0.0        0.0         1.0         0.0          0.0

In [10]:
# Danny Tarkanian
liar_train[liar_train['speaker']=='danny-tarkanian'][liar_speaker_headers].drop_duplicates()

speaker  speaker_job speaker_us_state speaker_affiliation  \
id                                                                              
1815.json   danny-tarkanian  Businessman           Nevada          republican   
12280.json  danny-tarkanian          NaN           Nevada          republican   

            speaker_bt  speaker_f  speaker_ht  speaker_mt  speaker_pof  
id                                                                      
1815.json          0.0        0.0         0.0         1.0          0.0  
12280.json         1.0        0.0         1.0         0.0          0.0

In [11]:
# Robert Puente
liar_train[liar_train['speaker']=='robert-puente'][liar_speaker_headers].drop_duplicates()

speaker                    speaker_job speaker_us_state  \
id                                                                          
10887.json  robert-puente                            NaN              NaN   
8784.json   robert-puente  CEO, San Antonio Water System            Texas   

           speaker_affiliation  speaker_bt  speaker_f  speaker_ht  speaker_mt  \
id                                                                              
10887.json                none         0.0        0.0         1.0         0.0   
8784.json             democrat         0.0        1.0         0.0         0.0   

            speaker_pof  
id                       
10887.json          0.0  
8784.json           0.0

In [12]:
# Replace NaNs with actual values
liar_train['speaker_affiliation'].mask(liar_train['speaker_affiliation'] == 'none', inplace=True)

for speaker in ['danny-tarkanian', 'kasim-reed', 'robert-puente']:
    liar_train.loc[liar_train['speaker']==speaker, liar_speaker_headers] = \
        liar_train.loc[liar_train['speaker']==speaker, liar_speaker_headers].fillna(method='bfill')
    liar_train.loc[liar_train['speaker']==speaker, liar_speaker_headers] = \
        liar_train.loc[liar_train['speaker']==speaker, liar_speaker_headers].fillna(method='ffill')

In [36]:
# Replace NaNs with empty strings
for header in liar_column_headers:
    if header not in ['id', 'label', 'statement']:
        liar_train[header] = liar_train[header].fillna('')

liar_train.head()

label                                          statement  \
id                                                                           
2635.json         false  Says the Annies List political group supports ...   
10540.json    half-true  When did the decline of coal start? It started...   
324.json    mostly-true  Hillary Clinton agrees with John McCain "by vo...   
1123.json         false  Health care reform legislation is likely to ma...   
9028.json     half-true  The economic turnaround started at the end of ...   

                                      subjects         speaker  \
id                                                               
2635.json                             abortion    dwayne-bohac   
10540.json  energy,history,job-accomplishments  scott-surovell   
324.json                        foreign-policy    barack-obama   
1123.json                          health-care    blog-posting   
9028.json                         economy,jobs   charlie-crist   

                     speaker_job speaker_us_state speaker_affiliation  \
id                                                                      
2635.json   State representative            Texas          republican   
10540.json        State delegate         Virginia            democrat   
324.json               President         Illinois            democrat   
1123.json                                                        none   
9028.json                                 Florida            democrat   

           speaker_bt speaker_f speaker_ht speaker_mt speaker_pof  \
id                                                                  
2635.json           0         1          0          0           0   
10540.json          0         0          1          1           0   
324.json           70        71        160        163           9   
1123.json           7        19          3          5          44   
9028.json          15         9         20         19           2   

                        context  
id                               
2635.json              a mailer  
10540.json      a floor speech.  
324.json                 Denver  
1123.json        a news release  
9028.json   an interview on CNN

<h2>Combine Features for Passing into Neural Network</h2>
<br><br>
We must now combine all the meta-data into the sentence features. (Reference: <a href = "https://www.kaggle.com/code/therealcyberlord/fake-news-detection-using-rnn/notebook">link</a>) Then, we will create batch-wise lookup tables of these sentences, during training, to pass as the input to our capsule networks.

In [130]:
X_train = liar_train
# news_df.drop('title', axis=1, inplace=True)

for header in liar_column_headers:
    if header not in ['id', 'label']:
        X_train['statement'] = X_train['statement'] + str(X_train[header])

In [131]:
drop_cols = [header for header in liar_column_headers if header not in ['id', 'label', 'statement']]

In [132]:
# X_train.drop(columns = ['id', 'subjects', 'speaker', 'speaker_job', 'speaker_us_state', 'speaker_affiliation', 'speaker_bt', 'speaker_f', 'speaker_ht', 'speaker_mt', 'speaker_pof', 'context'])
X_train = X_train.drop(columns = drop_cols)
X_train.head()

label                                          statement
id                                                                        
2635.json         false  Says the Annies List political group supports ...
10540.json    half-true  When did the decline of coal start? It started...
324.json    mostly-true  Hillary Clinton agrees with John McCain "by vo...
1123.json         false  Health care reform legislation is likely to ma...
9028.json     half-true  The economic turnaround started at the end of ...

In [133]:
Y_train = X_train['label']
X_train = X_train.drop('label', axis = 1)
X_train.head()

statement
id                                                           
2635.json   Says the Annies List political group supports ...
10540.json  When did the decline of coal start? It started...
324.json    Hillary Clinton agrees with John McCain "by vo...
1123.json   Health care reform legislation is likely to ma...
9028.json   The economic turnaround started at the end of ...

In [134]:
Y_train.head()

id
2635.json           false
10540.json      half-true
324.json      mostly-true
1123.json           false
9028.json       half-true
Name: label, dtype: object

In [135]:
labelEnc = LabelEncoder()
labelEnc.fit(Y_train)
labelEnc.classes_

array(['barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire',
       'true'], dtype=object)

In [136]:
labels = ['barely-true', 'false', 'half-true', 'mostly-true', 'pants-fire', 'true']

In [137]:
Y_train = labelEnc.transform(Y_train)

In [140]:
Y_train.shape

(10240,)

In [138]:
oneHotEnc = OneHotEncoder()
oneHotEnc.fit(Y_train)

/common/home/ncp67/Documents/capsule_fake_news_detection/venv/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


OneHotEncoder(categorical_features='all', dtype=<type 'float'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [143]:
oneHotEnc.categories__

AttributeError: 'OneHotEncoder' object has no attribute 'categories__'

<h2>Load GloVe</h2>

In [14]:
from gensim.models import KeyedVectors, Doc2Vec
from gensim.scripts.glove2word2vec import glove2word2vec

In [17]:
glove_file = './data/glove.6B.300d.txt'
tmp_file = './data/glovetmp.txt'

if not os.path.isfile(tmp_file):
    _ = glove2word2vec(glove_file, tmp_file)

glove_model = KeyedVectors.load_word2vec_format(tmp_file)

<h2>Convert GLoVe Model into an Embedding Matrix in Tensorflow</h2>
<br><br>
We now convert Glove Model into an embedding matrix in tensorflow. Reference: <a href = "https://stackoverflow.com/questions/53353978/how-to-project-my-word2vec-model-in-tensorflow">link</a>
<br><br>
This embedding matrix will later be used to create our embedding lookup tables, as implemented by Yang et. al.

In [20]:
vec_size = glove_model.vector_size
vocab_size = len(glove_model.vocab)

# Create the embedding matrix where words are indexed alphabetically
embedding_mat = np.zeros(shape=(vocab_size, vec_size), dtype='int32')
for idx, word in enumerate(sorted(glove_model.vocab)):
    embedding_mat[idx] = glove_model.get_vector(word)

# Setup the embedding matrix for tensorflow
# Static embeddings, i.e., non-trainable embeddings for short_text_capsule_model
static_embeddings = tf.Variable(embedding_mat, trainable = False)

# Non-static embeddings, i.e. trainable embeddings for long_text_capsule_model
nonstatic_embeddings = tf.Variable(embedding_mat, trainable = True)

In [ ]:
# Create empty lookup tables (Will be filled batch-wise during training)
max_sent = len(liar_train[0])

In [ ]:
class BatchGenerator(object):
    """Generate and hold batches."""
    def __init__(self, dataset,label, batch_size,input_size, is_shuffle=True):
      self._dataset = dataset
      self._label = label
      self._batch_size = batch_size    
      self._cursor = 0      
      self._input_size = input_size      
      
      if is_shuffle:
          index = np.arange(len(self._dataset))
          np.random.shuffle(index)
          self._dataset = np.array(self._dataset)[index]
          self._label = np.array(self._label)[index]
      else:
          self._dataset = np.array(self._dataset)
          self._label = np.array(self._label)
    def next(self):
      if self._cursor + self._batch_size > len(self._dataset):
          self._cursor = 0
      """Generate a single batch from the current cursor position in the data."""      
      batch_x = self._dataset[self._cursor : self._cursor + self._batch_size,:]
      batch_y = self._label[self._cursor : self._cursor + self._batch_size]
      self._cursor += self._batch_size
      return batch_x, batch_y